# GPUs

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ARCTraining/swd6_hpp/blob/master/docs/06_GPUs.ipynb)

```{note}
This GPU lesson focuses primarily on NVIDIA CUDA which is a proprietary solution, and that there are open-source alternatives such as OpenCL.
However, at present CUDA is the most used platform for GPU programming and therefore is included in this course.
**Please note this means the following code will not run on Apple Macs at they do not have compatible hardware**
```

```{admonition} Tip
:class: tip 
If you're in COLAB or have a local CUDA GPU, you can follow along with this section (i.e., uncomment the GPU code bits).

For those in COLAB, ensure the session is using a GPU by going to: Runtime > Change runtime type > Hardware accelerator = GPU.
```

[GPUs (Graphics Processing Units)](https://en.wikipedia.org/wiki/Graphics_processing_unit) are optimised for numerical operations, while [CPUs (central processing units)](https://en.wikipedia.org/wiki/Central_processing_unit) perform general computation.

Originally, GPUs handled computer graphics. However, they are now used to do a wide range of computations too. Hence, the term [General Purpose GPU (GPGPU)](https://en.wikipedia.org/wiki/General-purpose_computing_on_graphics_processing_units).  

GPU hardware is designed for data parallelism, where high throughputs are achieved when the GPU is computing the same operations on many different elements at once.

You could use other [types of accelerators](https://en.wikipedia.org/wiki/Hardware_acceleration) too, though we're not going to cover those here.  

## [Numba for CUDA GPUs](http://numba.pydata.org/numba-doc/latest/cuda/index.html)

Earlier we covered how Numba works on single CPUs with [`@njit`](https://numba.readthedocs.io/en/stable/glossary.html#term-nopython-mode) and multiple CPUs with `parallel = True`.

As a recap:

In [1]:
import numpy as np
from numba import njit

In [2]:
x = np.arange(1.0e7, dtype=np.int64)

So, for a single CPU:

In [3]:
@njit
def my_serial_function_for_cpu(x):
    return np.cos(x) ** 2 + np.sin(x) ** 2

In [4]:
%%timeit
my_serial_function_for_cpu(x)

137 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


And, for multiple CPUs:

In [5]:
@njit(parallel=True)
def my_parallel_function_for_cpu(x):
    return np.cos(x) ** 2 + np.sin(x) ** 2

In [6]:
%%timeit
my_parallel_function_for_cpu(x)

32.6 ms ± 7.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


```{note}
Here we used `njit` as this automates the parallelisation process.

This is in contrast to `vectorize` where manual effort is required for parallelisation.  
```

### `vectorize` for GPUs

Numba also works on [CUDA](https://developer.nvidia.com/how-to-cuda-python) GPUs using [`@vectorize`](https://numba.pydata.org/numba-doc/latest/user/vectorize.html) or [`@cuda.jit`](https://numba.readthedocs.io/en/stable/cuda/kernels.html).

This is suitable for bigger data sizes (> 1 MB) and high compute intensities.

This adds additional overhead due to moving data to and from GPUs ([memory management](https://numba.pydata.org/numba-doc/dev/cuda/memory.html)).

Similar to our examples in the compiler lesson, we need to specify the types and target in the signature (i.e., the decorator arguments).

Here, the types are specificed slightly differently i.e., output types(input types).

```{attention}
Not all NumPy code will work on the GPU. In the following example, we will need to use the `math` library instead.
```

In [7]:
import math

from numba import float32, vectorize

In [8]:
x = np.arange(1.0e7, dtype=np.float32)

In [9]:
@vectorize(["float32(float32)"], target="cuda")
def my_serial_function_for_gpu(x):
    return math.cos(x) ** 2 + math.sin(x) ** 2

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [ ]:
# %%timeit
# my_serial_function_for_gpu(x)

Numba also supports generalized ufuncs (covered in the compiler lesson) on the GPU using [`guvectorize`](http://numba.pydata.org/numba-doc/latest/cuda/ufunc.html#generalized-cuda-ufuncs).

### Custom CUDA kernels

Kernel functions are GPU functions called from CPU code.

Kernels cannot explicitly return a value. Instead, all result data must be written to an array passed to the function (e.g., called `out`). This array can then be transferred back to the CPU.

Kernels work over a grid of threads. This grid needs to be defined in terms of the number of blocks in the grid and the number of threads per block. The indices of this grid are used to add values to the `out` array. The indices can be found using [`cuda.grid()`](https://numba.pydata.org/numba-doc/dev/cuda-reference/kernel.html#numba.cuda.grid).

CUDA kernels are compiled using the [`numba.cuda.jit`](https://numba.pydata.org/numba-doc/dev/cuda-reference/kernel.html#numba.cuda.jit) decorator.

```{note}
`numba.cuda.jit` is different to `numba.jit`, which is for CPUs.
```

In [ ]:
# from numba import cuda

In [ ]:
# print(cuda.gpus)

This should return a message similar to:  
<Managed Device 0>.

You can also run the bash command `nvidia-smi` within the IPython cell:

In [ ]:
# !nvidia-smi

This returns something like the table below. This shows we have access to a [NVIDIA Tesla T4 GPU](https://www.nvidia.com/en-gb/data-center/tesla-t4/).

```bash
Tue Feb 22 13:59:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |    144MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
+-----------------------------------------------------------------------------+
```

So, a simple example to add two numbers together:

In [ ]:
# @cuda.jit
# def add_kernel(x, y, out):
#     index = cuda.grid(1)
#     out[index] = x[index] + y[index]

Let's define some input variables:

In [ ]:
# n = 4096
# x = np.arange(n).astype(np.int32) # [0...4095] on the host (CPU)
# y = np.ones_like(x)               # [1...1] on the host (CPU)

Now, let's move these input variables from the host (CPU) to the device (GPU) for the work:

In [ ]:
# x_on_device = cuda.to_device(x)
# y_on_device = cuda.to_device(y)
# out_on_device = cuda.device_array_like(x_on_device)

Now, we [choose the block size](https://numba.pydata.org/numba-doc/latest/cuda/kernels.html#choosing-the-block-size), by defining how many blocks are in the grid and how many threads are in each of those blocks.

These two numbers multipled together is the size of the grid (for our 1D example).

![cuda_grid.png](images/cuda_grid.png)  

*[Image source](https://developer.nvidia.com/blog/cuda-refresher-cuda-programming-model/)*

Some rules of thumb are:
- Blocks per grid should be a multiple of 32.
- Threads per block should be a multiple of 128.

In [ ]:
# blocks_per_grid = 32
# threads_per_block = 128

Now, we can call the kernel function.

First, add the grid size arguments.

Then, we pass the input/output variables as arguments to the function.

In [ ]:
# add_kernel[blocks_per_grid, threads_per_block](x_on_device, y_on_device, out_on_device)

As these CUDA kernels don't return a value, we can synchronise the device (GPU) back to the host (CPU) to get the result back.

In [ ]:
# cuda.synchronize()
# print(out_on_device.copy_to_host())
# # Should be [   1    2    3 ... 4094 4095 4096]

For more information on CUDA, see the training courses:

- [HPC5: Introduction to GPU programming with CUDA](https://arc.leeds.ac.uk/training/courses/hpc5/)
- NVIDIA workshop on [Fundamentals of Accelerated Computing with CUDA Python](https://www.nvidia.com/en-us/training/instructor-led-workshops/fundamentals-of-accelerated-computing-with-cuda-python/)
    - Detailed look at [custom CUDA kernels](https://numba.pydata.org/numba-doc/dev/cuda/kernels.html) and [GPU memory management](https://numba.pydata.org/numba-doc/dev/cuda/memory.html).

## [RAPIDS](https://developer.nvidia.com/rapids)

RAPIDS is a range of accelerated data science libraries from NVIDIA.

There are a wide variety of tools matching up to familiar libraries:

- Arrays and matrices
  - [cuPy](https://cupy.dev/) for NumPy and SciPy
- Tabular data
    - [cuDF](https://docs.rapids.ai/api/cudf/stable/) for Pandas
- Machine learning
    - [cuML](https://docs.rapids.ai/api/cuml/stable/) for scikit-learn
    - [XGBoost](https://rapids.ai/xgboost.html) on GPUs
- Graphs and networks
    - [cuGraph](https://docs.rapids.ai/api/cugraph/stable/) for [NetworkX](https://networkx.org/)
- Multiple GPUs
    - [Dask with CUDA](https://rapids.ai/dask.html), cuDF, cuML, and others.
    - [Dask-MPI with GPUs](http://mpi.dask.org/en/latest/gpu.html)

### [cuPy](https://cupy.dev/)

**NumPy for the CPU**

In [10]:
import numpy as np

In [11]:
x_cpu = np.random.rand(1_000, 1_000)
y_cpu = np.random.rand(1_000, 1_000)

In [12]:
%%timeit
z_cpu = np.dot(x_cpu, y_cpu)

33.2 ms ± 9.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**CuPy for the GPU**

In [ ]:
# import cupy as cp

In [ ]:
# x_gpu = cp.random.rand(1_000, 1_000)
# y_gpu = cp.random.rand(1_000, 1_000)

In [ ]:
# %%timeit
# z_gpu = cp.dot(x_gpu, y_gpu)

You can move arrays between the CPU and GPU as follows:

In [ ]:
# z_gpu = cp.asarray(z_cpu)  # from cpu to gpu

In [ ]:
# z_cpu = cp.asnumpy(z_gpu)  # from gpu to cpu

For more information on RAPIDS, see the training courses:

- NVIDIA workshop on [Fundamentals of Accelerated Data Science (RAPIDS)](https://www.nvidia.com/en-us/training/instructor-led-workshops/fundamentals-of-accelerated-data-science/).

## Diagnostics

Similar to the Dask Dashboard, NVIDIA has a GPU Dashboard called [`NVDashboard`](https://github.com/rapidsai/jupyterlab-nvdashboard).

These real-time diagnostics are provided via a Bokeh server and a Jupyter Lab extension.  

They are a great way to manage your GPU utilisation, resources, throughput, and more.  

More information is [here](https://developer.nvidia.com/blog/gpu-dashboards-in-jupyter-lab/).

![SegmentLocal](images/NVIDIA_GPUDashboard.gif "segment")

*[Image source](https://developer.nvidia.com/blog/gpu-dashboards-in-jupyter-lab/)*

## [JAX](https://jax.readthedocs.io/en/latest/index.html)

JAX enables:

- NumPy on the CPU and GPU (via [XLA, Accelerated Linear Algebra](https://www.tensorflow.org/xla), a compiler for linear algebra).
- Automatic differentiation of native Python and NumPy code (via [Autograd](https://github.com/hips/autograd)).
- [`Jit`](https://jax.readthedocs.io/en/latest/_autosummary/jax.jit.html#jax.jit) compiler to speed up code.
- Automatic vectorisation through [`vmap`](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html#jax.vmap).

### JAX can replace NumPy for GPUs

If it can't find a GPU, then it will fall back to the CPU.

In [13]:
import jax
import jax.numpy as jnp
import numpy as np

In [14]:
x_np = np.arange(10)
print(type(x_np))
x_np

<class 'numpy.ndarray'>


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [15]:
x_jnp = jnp.arange(10)
print(type(x_jnp))
x_jnp

2022-02-25 16:02:15.867936: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-25 16:02:15.867960: W external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


<class 'jaxlib.xla_extension.DeviceArray'>


DeviceArray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

However, there are some differences between JAX and NumPy.

For example, [JAX arrays are immutable](https://jax.readthedocs.io/en/latest/notebooks/thinking_in_jax.html#jax-vs-numpy) (i.e., you can't change them once their made).

In [16]:
x_np[0] = 10
x_np

array([10,  1,  2,  3,  4,  5,  6,  7,  8,  9])

In [17]:
try:
    x_jnp[0] = 10
    print(x_jnp)
except TypeError:
    print("Sorry, you can't change JAX arrays once their made.")

Sorry, you can't change JAX arrays once their made.


Instead, you can create a copy with the change:

In [18]:
updated_x_jnp = x_jnp.at[0].set(10)
updated_x_jnp

DeviceArray([10,  1,  2,  3,  4,  5,  6,  7,  8,  9], dtype=int32)

Also, [random arrays are created differently](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#random-numbers):

In [23]:
x_np = np.random.normal(size=(3_000, 3_000))

In [32]:
from jax import random

In [33]:
random_key = random.PRNGKey(0)
x_jnp = random.normal(random_key, (3_000, 3_000), dtype=jnp.float32)

Now, you could multiply arrays:

In [25]:
%timeit np.dot(x_np, x_np.T)

242 ms ± 18.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit jnp.dot(x_jnp, x_jnp.T).block_until_ready()

187 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


By default, this will transfer data from the host (CPU) to the device (GPU).

To avoid this and [put data](https://jax.readthedocs.io/en/latest/_autosummary/jax.device_put.html#jax.device_put) onto the device (GPU):

In [27]:
from jax import device_put

In [28]:
x_jnp = device_put(x_jnp)

In [29]:
%timeit jnp.dot(x_jnp, x_jnp.T).block_until_ready()

188 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In general, the speed comparison between JAX and NumPy is complicated and depends on a variety of things ([read more here](https://jax.readthedocs.io/en/latest/faq.html#faq-jax-vs-numpy)).

### [`@jit`](https://jax.readthedocs.io/en/latest/_autosummary/jax.jit.html#jax.jit) compiler

Let's see an example of using the JAX `@jit` compiler to speed up a function:

In [30]:
from jax import jit

In [34]:
x = random.normal(random_key, (1_000_000,))

Here were using an example for the [SELU (Scaled Exponential Linear Unit)](https://arxiv.org/pdf/1706.02515v5.pdf) activation function (*don't worry what this is*).

In [35]:
def slow_selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

In [38]:
%%timeit
slow_selu(x).block_until_ready()

1.12 ms ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [52]:
fast_selu = jit(slow_selu)

Remember from our compiler lesson, that the first call to a JIT-decorated function compiles it:

In [53]:
%%timeit -n 1 -r 1
fast_selu(x).block_until_ready()

25 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Then all subsequent calls to it use the cached, fast version:

In [54]:
%%timeit -n 1 -r 1
fast_selu(x).block_until_ready()

489 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Automatic vectorisation with [`vmap()`](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html#jax.vmap)

`vmap` maps a function over array axes.

This pushes the map loop lower down for better performance.

Let's see an example of multiplying a matrix by a batch of vectors (*don't worry what this function does, just focus on the JAX bits*):

In [42]:
from jax import vmap

In [43]:
matrix = random.normal(random_key, (150, 100))
batch_vectors = random.normal(random_key, (10, 100))

In [44]:
def multiplying_matrix_by_vector(vector):
    return jnp.dot(matrix, vector)

So, first apply this function in a simple batch:

In [55]:
def simple_batch(batch_vectors):
    return jnp.stack([multiplying_matrix_by_vector(vector) for vector in batch_vectors])

In [56]:
%timeit simple_batch(batch_vectors).block_until_ready()

726 µs ± 32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Now, let's instead use a `vmap` batch to map the function over the matrix:

In [57]:
def vmap_batch(batch_vectors):
    return vmap(multiplying_matrix_by_vector)(batch_vectors)

In [58]:
%timeit vmap_batch(batch_vectors).block_until_ready()

224 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


And, let's get even more performance by combining this with the `jit` compiler:

In [59]:
@jit
def faster_vmap_batch(batch_vectors):
    return vmap(multiplying_matrix_by_vector)(batch_vectors)

In [60]:
%timeit faster_vmap_batch(batch_vectors).block_until_ready()

23.1 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


There is lots more useful information in the [documentation](https://jax.readthedocs.io/en/latest/index.html), such as [a range of tutorials](https://jax.readthedocs.io/en/latest/jax-101/index.html).

## Exercises

```{admonition} Exercise 1

In general, what kind of tasks are GPUs faster than CPUs for, and why?

```

```{admonition} Exercise 2

What Numba decorators can you use to offload a function to GPUs?

```

```{admonition} Exercise 3

How would you vectorize the the following function for GPUs?  

`def my_serial_function_for_gpu(x):`  
`    return math.cos(x) ** 2 + math.sin(x) ** 2`  

```

```{admonition} Exercise 4

What are ways you can check if your Python environment has access to a GPU?

```

```{admonition} Exercise 5

If you wanted to do NumPy style work on GPUs, could you use:

- cuPy
- JAX

```

## {ref}`Solutions <gpus>`

## Key Points

```{important}

- [x] _Use [CUDA/Numba](https://developer.nvidia.com/how-to-cuda-python), [RAPIDS](https://developer.nvidia.com/rapids), and [JAX](https://jax.readthedocs.io/en/latest/index.html) to write custom data science code for CUDA GPUs._

```

## Further information

### Good practises

- Test out ideas on CPUs first, before moving to expensive GPUs.
- Consider whether the calculation is worth the additional overhead of sending data to and from the GPU.
- Minimise data transfers between the host (CPU) and the device (GPU).

### Other options

- [pycuda](https://documen.tician.de/pycuda/)
    - An alternative to Numba for accessing NVIDIA's CUDA GPUs.
- [cuNumeric](https://developer.nvidia.com/cunumeric)
    - A swap-out replacement for NumPy from NVIDIA for distributed GPUs.
    - Early stages of development.
    - Requires a separate interperter to run (Legate).
- Many libraries can use GPUs automatically if they can detect one e.g., [`TensorFlow`](https://www.tensorflow.org/install/gpu) and [`PyTorch`](https://pytorch.org/docs/stable/notes/cuda.html).

### Resources

- [CuPy - Sean Farley](https://www.youtube.com/watch?v=_AKDqw6li58), PyBay 2019.  
- [cuDF - Mark Harris](https://www.youtube.com/watch?v=lV7rtDW94do), PyCon AU 2019.  